# Part II.3: Kiểm tra giả định của mô hình AR(1)
## SCENARIO 1: AR(1) trên dữ liệu gốc (Original Data)

**Mục tiêu**: Fit AR(1) trực tiếp lên dữ liệu CO2 gốc (không xử lý), kiểm tra các giả định

In [ ]:
# Import thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Thiết lập style cho đồ thị
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Đọc dữ liệu CO2 monthly
df = pd.read_csv('group1_co2_monthly.csv')
df['index'] = pd.to_datetime(df['index'])
df.set_index('index', inplace=True)
df.index.freq = 'M'  # Set monthly frequency

print(f"Số quan sát: {len(df)}")
print(f"Khoảng thời gian: {df.index[0]} đến {df.index[-1]}")
print(f"\nMean: {df['co2'].mean():.4f}")
print(f"Std: {df['co2'].std():.4f}")
print(f"\n5 giá trị đầu:")
print(df.head())

## 0. Exploratory Data Analysis (EDA)

### 0.1. Visualize Time Series

In [ ]:
# Plot time series
plt.figure(figsize=(14, 5))
plt.plot(df.index, df['co2'], linewidth=1.5, color='blue')
plt.title('Monthly CO2 Concentration (1958-2001)', fontsize=14, fontweight='bold')
plt.xlabel('Time', fontsize=12)
plt.ylabel('CO2 (ppm)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 Quan sát: Dữ liệu có TREND tăng rõ rệt + SEASONAL pattern lặp lại hàng năm")

### 0.2. Time Series Decomposition

Phân tích chuỗi thời gian thành 3 thành phần: **Trend** + **Seasonal** + **Residual**

In [ ]:
# Decompose time series
decomposition = seasonal_decompose(df['co2'], model='additive', period=12)

fig, axes = plt.subplots(4, 1, figsize=(14, 10))

# Original
axes[0].plot(df.index, df['co2'], color='blue')
axes[0].set_ylabel('Original', fontsize=11)
axes[0].set_title('Time Series Decomposition', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Trend
axes[1].plot(decomposition.trend.index, decomposition.trend, color='red')
axes[1].set_ylabel('Trend', fontsize=11)
axes[1].grid(True, alpha=0.3)

# Seasonal
axes[2].plot(decomposition.seasonal.index, decomposition.seasonal, color='green')
axes[2].set_ylabel('Seasonal', fontsize=11)
axes[2].grid(True, alpha=0.3)

# Residual
axes[3].plot(decomposition.resid.index, decomposition.resid, color='purple')
axes[3].set_ylabel('Residual', fontsize=11)
axes[3].set_xlabel('Time', fontsize=11)
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Phân tích:")
print("  • Trend: Tăng tuyến tính rõ rệt từ ~315 ppm (1958) lên ~370 ppm (2001)")
print("  • Seasonal: Chu kỳ 12 tháng lặp lại đều đặn, biên độ ~6-7 ppm")
print("  • Residual: Phần còn lại sau khi loại bỏ trend và seasonal")
print("\n⚠️ Vấn đề: AR(1) không có cơ chế xử lý trend và seasonality!")

### 0.3. ACF & PACF của dữ liệu gốc

Kiểm tra cấu trúc tự tương quan trước khi fit model

In [ ]:
# ACF và PACF của original data
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

plot_acf(df['co2'], lags=40, ax=axes[0], alpha=0.05)
axes[0].set_title('ACF of Original CO2 Data', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=12)
axes[0].set_ylabel('ACF', fontsize=12)
axes[0].grid(True, alpha=0.3)

plot_pacf(df['co2'], lags=40, ax=axes[1], alpha=0.05)
axes[1].set_title('PACF of Original CO2 Data', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=12)
axes[1].set_ylabel('PACF', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Quan sát:")
print("  • ACF: Giảm rất chậm (slow decay) → Dấu hiệu của NON-STATIONARY (có trend)")
print("  • ACF: Có peaks tại lag 12, 24, 36 → Seasonality rõ rệt")
print("  • PACF: Spike mạnh tại lag 1, sau đó có peaks nhỏ hơn")
print("\n⚠️ Kết luận: Dữ liệu NON-STATIONARY với SEASONAL pattern → AR(1) sẽ không phù hợp")

## 1. Fit mô hình AR(1) - ARIMA(1,0,0)

Mô hình: $Y_t = c + \phi Y_{t-1} + e_t$

- **ARIMA(1,0,0)** = AR(1) không differencing
- Fit trực tiếp lên dữ liệu gốc

In [ ]:
# Fit AR(1) model bằng ARIMA(1,0,0)
model_ar1 = ARIMA(df['co2'], order=(1,0,0)).fit()

print("="*80)
print("MÔ HÌNH AR(1) - KẾT QUẢ ƯỚC LƯỢNG")
print("="*80)
print(model_ar1.summary())

In [ ]:
# Trích xuất phần dư (residuals)
residuals = model_ar1.resid

print("\nTHAM SỐ ƯỚC LƯỢNG:")
print(f"  φ̂ (AR coefficient): {model_ar1.params[1]:.6f}")
print(f"  ĉ (Constant): {model_ar1.params[0]:.6f}")
print(f"  n (observations): {len(residuals)}")
print(f"\n⚠️ Chú ý: |φ̂| = {abs(model_ar1.params[1]):.6f} ≈ 1 → Gần unit root (non-stationary)")

## 2. Kiểm tra Giả định 1: Phần dư có trung bình = 0 và phương sai không đổi

**Yêu cầu**: 
- $E(\hat{e}_t) = 0$
- $Var(\hat{e}_t)$ không thay đổi theo thời gian

In [ ]:
# Tính mean và std của residuals
mean_resid = residuals.mean()
std_resid = residuals.std()

print("GIẢ ĐỊNH 1: ZERO MEAN & CONSTANT VARIANCE")
print("="*60)
print(f"Mean của phần dư: {mean_resid:.6f}")
print(f"Std của phần dư: {std_resid:.6f}")
print(f"\n✓ Mean ≈ 0? {abs(mean_resid) < 0.1} (|mean| = {abs(mean_resid):.6f})")
print(f"Status: {'PASS' if abs(mean_resid) < 0.1 else 'FAIL'} (ngưỡng: |mean| < 0.1)")

In [ ]:
# Vẽ biểu đồ phần dư theo thời gian và rolling std
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot 1: Residuals vs Time
axes[0].plot(residuals.index, residuals, linewidth=1, alpha=0.7, color='blue')
axes[0].axhline(0, color='red', linestyle='--', linewidth=2, label='Zero Line')
axes[0].axhline(mean_resid, color='green', linestyle='--', linewidth=2, label=f'Mean = {mean_resid:.4f}')
axes[0].fill_between(residuals.index, -2*std_resid, 2*std_resid, alpha=0.2, color='gray', label='±2σ')
axes[0].set_title('Assumption 1: Residuals vs Time', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Residuals', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: Rolling Standard Deviation (kiểm tra phương sai không đổi)
rolling_std = residuals.rolling(window=12).std()
axes[1].plot(rolling_std.index, rolling_std, linewidth=1.5, color='purple', label='Rolling Std (12 months)')
axes[1].axhline(std_resid, color='red', linestyle='--', linewidth=2, label=f'Overall Std = {std_resid:.4f}')
axes[1].set_title('Rolling Standard Deviation (Constant Variance Check)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Std Dev', fontsize=12)
axes[1].set_xlabel('Time', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/scenario1_assumption1.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Đồ thị đã được lưu: outputs/scenario1_assumption1.png")

## 3. Kiểm tra Giả định 2: Phần dư có phân phối chuẩn

**Kiểm định Shapiro-Wilk**:
- $H_0$: Phần dư có phân phối chuẩn
- $H_1$: Phần dư không có phân phối chuẩn
- Reject $H_0$ nếu p-value < 0.05

In [ ]:
# Shapiro-Wilk test
stat, p_value = stats.shapiro(residuals)

print("GIẢ ĐỊNH 2: NORMALITY TEST")
print("="*60)
print(f"Shapiro-Wilk Test:")
print(f"  Statistic: {stat:.4f}")
print(f"  p-value: {p_value:.4e}")

if p_value > 0.05:
    print(f"  ✓ p-value > 0.05 → Chấp nhận H₀ (Phần dư có phân phối chuẩn)")
else:
    print(f"  ✗ p-value < 0.05 → Bác bỏ H₀ (Phần dư KHÔNG có phân phối chuẩn)")

print(f"\nStatus: {'PASS' if p_value >= 0.05 else 'FAIL'}")

In [ ]:
# Vẽ Q-Q plot và Histogram
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Q-Q plot (kiểm tra phân phối chuẩn)
stats.probplot(residuals, dist="norm", plot=axes[0])
axes[0].set_title('Assumption 2: Q-Q Plot (Normality Check)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Histogram với đường cong chuẩn
axes[1].hist(residuals, bins=30, density=True, alpha=0.7, color='skyblue', edgecolor='black', label='Residuals')
mu, sigma = residuals.mean(), residuals.std()
x = np.linspace(residuals.min(), residuals.max(), 100)
axes[1].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label=f'Normal(μ={mu:.3f}, σ={sigma:.3f})')
axes[1].set_title('Histogram with Normal Curve', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Residuals', fontsize=12)
axes[1].set_ylabel('Density', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/scenario1_assumption2.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Đồ thị đã được lưu: outputs/scenario1_assumption2.png")

## 4. Kiểm tra Giả định 3: Phần dư không có tự tương quan

**Kiểm định Ljung-Box**:
- $H_0$: Không có tự tương quan ở tất cả các lag
- $H_1$: Có tự tương quan
- Reject $H_0$ nếu p-value < 0.05

In [ ]:
# Ljung-Box Test tại lag 10 và 20
lb_test = acorr_ljungbox(residuals, lags=[10, 20], return_df=True)

print("GIẢ ĐỊNH 3: NO AUTOCORRELATION TEST")
print("="*60)
print("\nLjung-Box Test Results:")
print(lb_test)

p_val_lag10 = lb_test.loc[10, 'lb_pvalue']
if p_val_lag10 > 0.05:
    print(f"\n✓ Lag 10: p-value={p_val_lag10:.4f} > 0.05 → Không có tự tương quan (Tốt)")
else:
    print(f"\n✗ Lag 10: p-value={p_val_lag10:.4f} < 0.05 → Có tự tương quan (Mô hình chưa tốt)")

In [ ]:
# Kiểm tra chi tiết 20 lags
lb_test_full = acorr_ljungbox(residuals, lags=20, return_df=True)
significant_lags = lb_test_full[lb_test_full['lb_pvalue'] < 0.05]

print(f"Số lags có p-value < 0.05: {len(significant_lags)}/20")
if len(significant_lags) > 0:
    print(f"Các lags significant: {significant_lags.index.tolist()}")
    print("\n⚠️ Phát hiện tự tương quan mạnh, đặc biệt tại lag 12, 24, 36 (seasonal pattern)")

print(f"\nStatus: {'PASS' if len(significant_lags) == 0 else 'FAIL'} (kỳ vọng: 0 lags significant)")

In [ ]:
# Vẽ ACF và PACF của residuals
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# ACF plot
plot_acf(residuals, lags=40, ax=axes[0], alpha=0.05, zero=False)
axes[0].set_title('Assumption 3: ACF of Residuals', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=12)
axes[0].set_ylabel('ACF', fontsize=12)
axes[0].grid(True, alpha=0.3)

# PACF plot
plot_pacf(residuals, lags=40, ax=axes[1], alpha=0.05)
axes[1].set_title('PACF of Residuals', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=12)
axes[1].set_ylabel('PACF', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/scenario1_assumption3.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Đồ thị đã được lưu: outputs/scenario1_assumption3.png")

## 5. Tổng kết kết quả kiểm tra giả định

In [ ]:
print("="*80)
print("TỔNG KẾT - SCENARIO 1: AR(1) ON ORIGINAL DATA")
print("="*80)

print("\n1. Zero Mean:")
print(f"   Mean = {mean_resid:.6f}")
print(f"   Status: {'✓ PASS' if abs(mean_resid) < 0.1 else '✗ FAIL'}")

print("\n2. Constant Variance:")
print(f"   Std Dev = {std_resid:.6f}")
print(f"   Status: ✓ PASS (visual inspection)")

print("\n3. Normality:")
print(f"   Shapiro-Wilk p-value = {p_value:.4e}")
print(f"   Status: {'✓ PASS' if p_value >= 0.05 else '✗ FAIL'}")

print("\n4. No Autocorrelation:")
print(f"   Significant lags: {len(significant_lags)}/20")
print(f"   Status: {'✓ PASS' if len(significant_lags) == 0 else '✗ FAIL'}")

print("\n" + "="*80)
print("KẾT LUẬN TỔNG QUÁT")
print("="*80)
print("""
Mô hình AR(1) trên dữ liệu CO2 gốc VI PHẠM nhiều giả định quan trọng:

• φ̂ ≈ 1 → Dữ liệu non-stationary (có trend)
• Phần dư KHÔNG có phân phối chuẩn (có outlier)
• Phần dư có tự tương quan MẠNH (20/20 lags significant)
• Phát hiện seasonal pattern rõ rệt tại lag 12, 24, 36

→ AR(1) KHÔNG PHÙ HỢP cho dữ liệu này vì không thể capture trend và seasonality

💡 KHUYẾN NGHỊ: Nên sử dụng mô hình SARIMA(p,d,q)(P,D,Q)[12] có khả năng:
   - Xử lý trend qua differencing (d)
   - Xử lý seasonality qua seasonal components (P,D,Q)
""")

print("="*80)